In [1]:
%%capture
pip install spacy-transformers


In [2]:
import tensorflow as tf
tf.test.gpu_device_name()

'/device:GPU:0'

In [3]:
import spacy
import json
import pandas as pd
import os
from tqdm import tqdm
from spacy.tokens import DocBin


In [4]:
from time import sleep
from IPython.display import clear_output, display

In [ ]:
# load drive 

'/content/drive/MyDrive/Colab Notebooks'

In [5]:
#load dataset 
%cd '/content/drive/MyDrive/Colab Notebooks/'


/content/drive/MyDrive/Colab Notebooks


In [6]:
#load dataset 

with open('dataset_model4.json','r') as file:
    temp_dataset= json.load(file)

In [ ]:
len(temp_dataset)

1960

In [7]:
dataset0 =[]
for idx,item in enumerate(temp_dataset):
  dataset0.append(tuple(temp_dataset[idx][0]))


In [ ]:
len(dataset0)

1960

In [ ]:
dataset0[2]

('306b waiting or loading restrictions 306b see county council replies 306c oneway driving 306c see county council replies 306d prohibition of driving 306d see county council replies 306e pedestrianisation 306e see county council replies 306f vehicle width or weight restriction 306f see county council replies 306g traffic calming works eg road humps 306g see county council replies 306h residents parking controls 306h see county council replies 306i minor road widening or improvement 306i see county council replies 306j pedestrian crossings 306j see county council replies 306k cycle tracks 306k see county council replies 306l bridge building 306I see county council replies 307a building works 307a no 307b environment 307b please see response to question 307 a 307c health and safety 307c please see response to question 307 a 307d housing 307d please see response to question 307 a 307e highways 307e please see response to question 307 a 307f public health 307f please see response to quest

In [ ]:
dataset0[0][0][38:198]

'38 has a local authority authorised in relation to the property any proceedings for the contravention of any provision contained in building regulations no howe'

In [8]:
db = DocBin() # create a DocBin object

In [9]:


nlp = spacy.blank("en") # load a new spacy model
doc_bin = DocBin() # create a DocBin object

In [10]:
import random

In [11]:
from spacy.util import filter_spans
count = 0 
for training_example  in tqdm(temp_dataset): 
    
  # if count>18500:
    # print(training_example)
    text = training_example[0][0]
    labels = training_example[0][1]['entities']
    # print(labels)
    
    doc = nlp.make_doc(text) 
    ents = []
    for start, end, label in labels:
        span = doc.char_span(start, end, label=label, alignment_mode="contract")
        
        if span is None:
            print("Skipping entity")
        else:
            ents.append(span)
    
    filtered_ents = filter_spans(ents)
    # print(filtered_ents)
    doc.ents = filtered_ents 
    # print(doc.ents)
    doc_bin.add(doc)
  # else:
  #   count+=1
doc_bin.to_disk("training_data.spacy") # save the docbin object

100%|██████████| 1103/1103 [00:01<00:00, 925.04it/s] 


In [12]:
doc.ents

(k an order revoking or modifying planning permission,
 I an order requiring discontinuance of use or alteration or removal of building or works)

In [13]:
!python -m spacy init fill-config base_config.cfg config.cfg

✔ Auto-filled config with all values
✔ Saved config
config.cfg
You can now add your data and train your pipeline:
python -m spacy train config.cfg --paths.train ./train.spacy --paths.dev ./dev.spacy


In [ ]:
!python -m spacy train config.cfg --output ./ --paths.train ./training_data.spacy --paths.dev ./training_data.spacy --gpu-id 0


ℹ Saving to output directory: .
ℹ Using GPU: 0

=========================== Initializing pipeline ===========================
[2022-12-15 09:27:23,876] [INFO] Set up nlp object from config
INFO:spacy:Set up nlp object from config
[2022-12-15 09:27:23,886] [INFO] Pipeline: ['transformer', 'ner']
INFO:spacy:Pipeline: ['transformer', 'ner']
[2022-12-15 09:27:23,890] [INFO] Created vocabulary
INFO:spacy:Created vocabulary
[2022-12-15 09:27:23,891] [INFO] Finished initializing nlp object
INFO:spacy:Finished initializing nlp object
Some weights of the model checkpoint at roberta-base were not used when initializing RobertaModel: ['lm_head.layer_norm.bias', 'lm_head.bias', 'lm_head.decoder.weight', 'lm_head.dense.weight', 'lm_head.dense.bias', 'lm_head.layer_norm.weight']
- This IS expected if you are initializing RobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining m

In [ ]:
nlp_ner = spacy.load("model-best")

In [ ]:
doc = nlp_ner("Antiretroviral therapy (ART) is recommended for all HIV-infected\
individuals to reduce the risk of disease progression.\nART also is recommended \
for HIV-infected individuals for the prevention of transmission of HIV.\nPatients \
starting ART should be willing and able to commit to treatment and understand the\
benefits and risks of therapy and the importance of adherence. Patients may choose\
to postpone therapy, and providers, on a case-by-case basis, may elect to defer\
therapy on the basis of clinical and/or psychosocial factors.")

colors = {"PATHOGEN": "#F67DE3", "MEDICINE": "#7DF6D9", "MEDICALCONDITION":"#FFFFFF"}
options = {"colors": colors} 

/usr/local/lib/python3.8/dist-packages/spacy/displacy/__init__.py:206: UserWarning: [W006] No entities to visualize found in Doc object. If this is surprising to you, make sure the Doc was processed using a model that supports named entity recognition, and check the `doc.ents` property manually if necessary.
  warnings.warn(Warnings.W006)


In [ ]:
text = "(c) a conservation area consent \
None \
(d) a certificate of lawfulness of existing use or development \
None \
(e) a certificate of lawfulness of proposed use or development \
None \
(f) a certificate of lawfulness of proposed works for listed buildings \
None \
(g) a heritage partnership agreement"

In [ ]:
text

'(c) a conservation area consent None (d) a certificate of lawfulness of existing use or development None (e) a certificate of lawfulness of proposed use or development None (f) a certificate of lawfulness of proposed works for listed buildings None (g) a heritage partnership agreement'

In [ ]:
import re

In [ ]:
line = re.sub('[.()-]', '', text)
print(line)

c a conservation area consent None d a certificate of lawfulness of existing use or development None e a certificate of lawfulness of proposed use or development None f a certificate of lawfulness of proposed works for listed buildings None g a heritage partnership agreement


In [ ]:
doc = nlp_ner(line)

colors = {"1.1_QUESTION": "#F67DE3", "1.1A_QUESTION": "#7DF6D9", "MEDICALCONDITION":"#FFFFFF"}
options = {"colors": colors} 

spacy.displacy.render(doc, style="ent", options= options, jupyter=True)